In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers, models, layers
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tf.keras.backend.clear_session()

In [3]:
import wandb
import optuna

from wandb.integration.keras import WandbMetricsLogger

wandb.require("core")
wandb.login()

wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.
wandb: Currently logged in as: emmdaz (emmdaz-zzz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
# Carga de los datos

ds = pd.read_csv("/global_house_purchase_dataset.csv")
# ds.head()

In [5]:
# print("NaN values: \n \n ", ds.isna().sum(),
#       "\n \n Types: \n \n", ds.dtypes)

In [6]:
# fig, axes = plt.subplots(2, 2, figsize = (10,10))
# axes = axes.flatten()

# sns.countplot(ds, x = "country", color = "purple", ax = axes[0])
# axes[0].tick_params(axis="x", rotation=90)
# axes[0].set_title("country")

# sns.countplot(ds, x = "city", color = "grey", ax = axes[1])
# axes[1].tick_params(rotation = 90)
# axes[1].set_title("city")

# sns.countplot(ds, x = "property_type", color = "orange", ax = axes[2])
# axes[2].tick_params(rotation = 90)
# axes[2].set_title("property_type")

# sns.countplot(ds, x = "furnishing_status", color = "green", ax = axes[3])
# axes[3].tick_params(rotation = 90)
# axes[3].set_title("furnishing_status")

# plt.tight_layout()
# plt.show() 

In [7]:
# One hot encoding

ds = pd.get_dummies(ds, columns = ["country", "city", "property_type", "furnishing_status"],
                            drop_first = False)
ds.drop("property_id", axis = 1, inplace = True)
ds = ds.astype("float32")

# ds.head()

In [8]:
# print("\n \n Types: \n \n", ds.dtypes)

In [9]:
x = ds.drop("decision", axis=1)
y = ds["decision"] 

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size = 0.3, random_state = 5)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 1/3, random_state = 5)

print("Train size:", len(X_train))
print("Validation size:", len(X_val))
print("Test size:", len(X_test))

Train size: 140000
Validation size: 40000
Test size: 20000


In [11]:
# Normalización de los datos

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [12]:
# X_train.shape 

In [13]:
def objective(trial):

    tf.keras.backend.clear_session()

    model = models.Sequential()

    # Optuna suggest first layer's neuron units and activation function
    
    n_units1 = trial.suggest_int("Layer_1_units", 32, 128)
    activation_1 = trial.suggest_categorical("Activation_1", ["relu", "sigmoid"])
    model.add(layers.Dense(n_units1, activation = activation_1, input_shape = (X_train.shape[1],)))

    # Optuna suggests number of layers
    
    n_layers = trial.suggest_int("n_layers", 1, 10)

    units_per_layer = [n_units1]
    activations_per_layer = [activation_1]
    dropout_per_layer = []
    dropout_percentage_per_layer = []
    
    # Optuna suggest regularizers for the hidden layers
    
    regularizer = trial.suggest_categorical("Regularizer", ["L1","L2","L1L2"])
    r_value = trial.suggest_float("regularizer_value", 1e-6, 1e-4, log = True)
    r_value2 = trial.suggest_float("regularizer_value_2", 1e-6, 1e-4, log = True)
    
    # Optuna suggests activation function, dropout and regularizers
    
    for i in range(n_layers):
        
        n_units = trial.suggest_int(f"Layer_{i+2}_units", 32, 128)
        activation = trial.suggest_categorical(f"Activation_{i+2}", ["relu", "sigmoid"])
        
        if regularizer == "L1":
            model.add(layers.Dense(n_units, activation = activation,
                                   kernel_regularizer = tf.keras.regularizers.L1(r_value)))
        elif regularizer == "L2":
            model.add(layers.Dense(n_units, activation = activation,
                                   kernel_regularizer = tf.keras.regularizers.L2(r_value)))
        elif regularizer == "L1L2":
            model.add(layers.Dense(n_units, activation = activation,
                                   kernel_regularizer = tf.keras.regularizers.L1L2(r_value, r_value2)))        
        
        dropout = trial.suggest_categorical(f"Dropout_L{i+2}", ["y", "n"])
        dropout_value = trial.suggest_float(f"Dropout_valueL{i+2}",0.2, 0.5)
        dropout_per_layer.append(dropout)
        
        if dropout == "y":
            model.add(layers.Dropout(dropout_value))
            dropout_percentage_per_layer.append(dropout_value)
        else:
            dropout_percentage_per_layer.append(0.0)     
        
        units_per_layer.append(n_units)
        activations_per_layer.append(activation)

    model.add(layers.Dense(1, activation="sigmoid"))
    
    units_per_layer.append(1)
    activations_per_layer.append("sigmoid")

    # Optuna suggests learning rate and optimizer
    
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "sgd", "rmsprop"])

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
    elif optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate = lr)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr)

    model.compile(
        optimizer = optimizer,
        loss = "binary_crossentropy",
        metrics = ["accuracy"]
    )

    wandb.init(
        project = "Global-House-Purchase-Decision-ANN-Trials-4",
        name = f"Trial_{trial.number}",
        reinit = True,
        config = {
            "n_layers": n_layers,
            "units_per_layer": units_per_layer,
            "activations_per_layer": activations_per_layer,
            "regularizer": regularizer,
            "dropout_per_layer": dropout_per_layer,
            "dropout_percentage_per_layer": dropout_percentage_per_layer,
            "learning_rate": lr,
            "optimizer": optimizer_name
        }
    )

    batch_size = trial.suggest_categorical("batch_size", [20, 40, 50, 100])

    history = model.fit(
        X_train, y_train,
        validation_data = (X_test, y_test),
        epochs = 100,
        batch_size = batch_size,
        verbose = 0, 
        callbacks = [WandbMetricsLogger(log_freq=5)]
    )

    val_loss = min(history.history["val_loss"])
    train_loss = min(history.history["loss"])

    # Penalize overfitting
    score = val_loss + 0.1 * (train_loss - val_loss)

    wandb.finish()

    return score


In [ ]:
study = optuna.create_study(study_name = "Proyecto", direction = "minimize")
study.optimize(objective, n_trials = 60)

[I 2025-10-14 17:13:20,357] A new study created in memory with name: Proyecto
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


batch/accuracy,▁▁▄▅▅▆▆▆▆▇▇▇▇▆▇▇▆▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇
batch/batch_step,▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▁▂▃▄▅▅▅▆▆▆▆▆▇▇▇▆▇▇█▇▇▇▇▇█▇▇▇██▇███▇█▇▇▇
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▃▂▆▂▁▆▄▅▇▃▅▂▂▅▅▇▇▇▅▇▇▇▇▆▇▆▃▇▅█▅▅▄▄▇▆▅▇▇▇
epoch/val_loss,▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁
batch/accuracy,0.999


[I 2025-10-14 17:17:12,171] Trial 0 finished with value: 0.0063767386600375175 and parameters: {'Layer_1_units': 111, 'Activation_1': 'relu', 'n_layers': 3, 'Regularizer': 'L2', 'regularizer_value': 1.2916814176250966e-06, 'regularizer_value_2': 5.826412325387065e-06, 'Layer_2_units': 63, 'Activation_2': 'sigmoid', 'Dropout_L2': 'n', 'Dropout_valueL2': 0.40200603606984886, 'Layer_3_units': 108, 'Activation_3': 'relu', 'Dropout_L3': 'y', 'Dropout_valueL3': 0.25941623252421053, 'Layer_4_units': 119, 'Activation_4': 'relu', 'Dropout_L4': 'y', 'Dropout_valueL4': 0.3241657875086515, 'learning_rate': 0.004253091471652316, 'optimizer': 'rmsprop', 'batch_size': 100}. Best is trial 0 with value: 0.0063767386600375175.


batch/accuracy,▁▂▁▃▄▄▄▅▇▄▅▇▅▅▆▅█▄▅▂▅▄▄▅▄▃▇▅▅▅▂▅▅▅▅▇▆▅▅▅
batch/batch_step,▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▁▂▂▂▁▂▂▁▁▁▁▁▁▁▁▂▂▁
epoch/accuracy,▁▇██████████████████████████████████████
epoch/epoch,▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▇▅▅▄▃▂▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,▆█▆▆▄▂▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/accuracy,0.76916


[I 2025-10-14 17:21:19,180] Trial 1 finished with value: 0.5413267016410828 and parameters: {'Layer_1_units': 36, 'Activation_1': 'sigmoid', 'n_layers': 7, 'Regularizer': 'L2', 'regularizer_value': 1.5326646337328253e-06, 'regularizer_value_2': 5.7471039015264604e-05, 'Layer_2_units': 104, 'Activation_2': 'relu', 'Dropout_L2': 'n', 'Dropout_valueL2': 0.4708159539323194, 'Layer_3_units': 44, 'Activation_3': 'sigmoid', 'Dropout_L3': 'n', 'Dropout_valueL3': 0.46641945951631153, 'Layer_4_units': 59, 'Activation_4': 'sigmoid', 'Dropout_L4': 'y', 'Dropout_valueL4': 0.3936646577955799, 'Layer_5_units': 66, 'Activation_5': 'sigmoid', 'Dropout_L5': 'n', 'Dropout_valueL5': 0.3611315193524218, 'Layer_6_units': 35, 'Activation_6': 'sigmoid', 'Dropout_L6': 'n', 'Dropout_valueL6': 0.22207984697004243, 'Layer_7_units': 116, 'Activation_7': 'sigmoid', 'Dropout_L7': 'y', 'Dropout_valueL7': 0.26328346910912936, 'Layer_8_units': 56, 'Activation_8': 'sigmoid', 'Dropout_L8': 'y', 'Dropout_valueL8': 0.27650

...

[I 2025-10-14 18:44:11,947] Trial 12 finished with value: 0.008109684032388032 and parameters: {'Layer_1_units': 112, 'Activation_1': 'relu', 'n_layers': 3, 'Regularizer': 'L2', 'regularizer_value': 9.640792685151539e-05, 'regularizer_value_2': 3.562141264026679e-05, 'Layer_2_units': 101, 'Activation_2': 'relu', 'Dropout_L2': 'y', 'Dropout_valueL2': 0.4396190427736295, 'Layer_3_units': 63, 'Activation_3': 'sigmoid', 'Dropout_L3': 'y', 'Dropout_valueL3': 0.20856992979468975, 'Layer_4_units': 103, 'Activation_4': 'relu', 'Dropout_L4': 'y', 'Dropout_valueL4': 0.4909668124477153, 'learning_rate': 0.00014614421588942666, 'optimizer': 'adam', 'batch_size': 20}. Best is trial 2 with value: 0.003312203101813793.


batch/accuracy,▁███████████████████████████████████████
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▄▅▅▆▆▇▇▇▇▇▇▇▇████▇█████████████████████
epoch/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▅▆▇▇▆▇▇▇▇█▇▄▇▇▇███▇▇▇▇██▇▇▇█▇▇▇▇██▇█▇██
epoch/val_loss,█▃▂▂▂▂▁▂▄▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂
batch/accuracy,0.99983


[I 2025-10-14 18:47:01,418] Trial 13 finished with value: 0.005313858587760479 and parameters: {'Layer_1_units': 109, 'Activation_1': 'relu', 'n_layers': 2, 'Regularizer': 'L2', 'regularizer_value': 1.6532255660370072e-05, 'regularizer_value_2': 3.9443862690715165e-05, 'Layer_2_units': 124, 'Activation_2': 'relu', 'Dropout_L2': 'y', 'Dropout_valueL2': 0.36882837881598907, 'Layer_3_units': 66, 'Activation_3': 'sigmoid', 'Dropout_L3': 'y', 'Dropout_valueL3': 0.28615540430876896, 'learning_rate': 0.0017813687043906456, 'optimizer': 'adam', 'batch_size': 100}. Best is trial 2 with value: 0.003312203101813793.


batch/accuracy,▁▄▄▄▅▅▄▅▅▆▅▆▆▆▅▆▆▆▆▆▆▆▆▆▆▆▆▅▆▇▆▆▆▆▆█▆▆▆▇
batch/batch_step,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▇██████████████████████████████████████
epoch/epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▄▄▆▆▇▆▇▇▆▇▇▆▇▆▇▇▇▇▇▇██▇▇█▇▇▇▆▇█▇▇███▇▇▇
epoch/val_loss,█▇▄▃▃▂▂▂▂▃▂▂▂▂▂▂▂▁▂▂▂▁▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁
batch/accuracy,0.9993


[I 2025-10-14 18:57:20,516] Trial 14 finished with value: 0.003745954972691834 and parameters: {'Layer_1_units': 55, 'Activation_1': 'relu', 'n_layers': 2, 'Regularizer': 'L1L2', 'regularizer_value': 1.6868256277540437e-05, 'regularizer_value_2': 2.4481958565803134e-05, 'Layer_2_units': 99, 'Activation_2': 'relu', 'Dropout_L2': 'n', 'Dropout_valueL2': 0.43562106911938375, 'Layer_3_units': 36, 'Activation_3': 'sigmoid', 'Dropout_L3': 'y', 'Dropout_valueL3': 0.2539135510636094, 'learning_rate': 0.00035406883716909845, 'optimizer': 'rmsprop', 'batch_size': 20}. Best is trial 2 with value: 0.003312203101813793.


In [ ]:
print("Número de pruebas terminadas: ", len(study.trials))

trial = study.best_trial

print("Mejor intento: ", trial)

print("Valor: ", trial.value)
print("Hiperparámetros: ", trial.params)

In [ ]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_slice
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank

In [ ]:
len(study.trials)

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
plot_param_importances(study)